# Hybrid model 

## This script does the following 

* Models the hybrid linear-linear coupled problem with varying gaussian source term
* Solves the hybrid model using domain decomposition

In [ ]:
print('\n Generating snapshots for the hybrid model')

start = time.time()
# Initialize snapshot matrix
s = np.zeros((ndofs_u1+ndofs_uI+ndofs_U2,nsamples))

# Initialize traction array and interface vector array
traction_array12 = np.zeros((nsamples,ndofs_uI))
u_I_list = np.zeros((nsamples,ndofs_uI))

for i in range(nsamples):
    
    print(i+1, "snapshots computed,",nsamples-i-1,"snapshots to go...", end="\r")
    
    f_2I = np.matmul(V_I.T,F_2I_array[i])
    f_I = np.matmul(V_I.T,F_I_array[i])

    if i == 0:
        
        # Create reduced matrices
        a_11 = np.matmul(np.matmul(V_1.T,A_11.toarray()),V_1)

        a_1I = np.matmul(np.matmul(V_1.T,A_1I.toarray()),V_I)
        a_I1 = np.matmul(np.matmul(V_I.T, A_I1.toarray()), V_1)

        a_II = np.matmul(np.matmul(V_I.T, A_II.toarray()), V_I)

        a_I2 = np.matmul(V_I.T,A_I2.toarray())
        a_2I = np.matmul(A_2I.toarray(),V_I)

        f_1 = np.matmul(V_1.T,F_1)
        f_1I = np.matmul(V_I.T,F_1I)
        a_II1 = np.matmul(np.matmul(V_I.T, A_II1.toarray()), V_I)

        a1_tilde = a_II1-np.matmul(np.matmul(a_I1,np.linalg.inv(a_11)),a_1I)
        f1_tilde = np.matmul(np.matmul(a_I1,np.linalg.inv(a_11)),f_1)-f_1I
    
    # Assemble again

    # Put A and F back together
    a = sps.bmat([[a_11, None, a_1I],[None, A_22,a_2I],[a_I1,a_I2,a_II]]).tocsc()
    f = sps.hstack([f_1,F_2_array[i],f_I]).tocsc()

    # Solve
    u = sps.linalg.spsolve(a,f.T)
    s[:,i] = u
    
    u_I = u[-ndofs_uI:]
    u_I_list[i] = u_I
    
    traction = np.matmul(a1_tilde,u_I)+f1_tilde
    traction_array12[i] = traction

    if i == nsamples-ntest:
        end = time.time()
        hybridtime = end-start

print("\n Finished.")